# **Objective:**

- Create regression models to predict college applications for different US unviersities

- What factors impact college application totals the most? 


**Import Necessary Packages and load the  dataset**


In [114]:
from sklearn import preprocessing as pp, model_selection, ensemble, linear_model as lm, metrics
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from tensorflow import keras
import sklearn

df = pd.read_csv('College.csv')
df

,Unnamed: 0,Private,Apps,Accept,Enroll,Top10perc,Top25perc,F.Undergrad,P.Undergrad,Outstate,Room.Board,Books,Personal,PhD,Terminal,S.F.Ratio,perc.alumni,Expend,Grad.Rate
0,Abilene Christian University,Yes,1660,1232,721,23,52,2885,537,7440,3300,450,2200,70,78,18.1,12,7041,60
1,Adelphi University,Yes,2186,1924,512,16,29,2683,1227,12280,6450,750,1500,29,30,12.2,16,10527,56
2,Adrian College,Yes,1428,1097,336,22,50,1036,99,11250,3750,400,1165,53,66,12.9,30,8735,54
3,Agnes Scott College,Yes,417,349,137,60,89,510,63,12960,5450,450,875,92,97,7.7,37,19016,59
4,Alaska Pacific University,Yes,193,146,55,16,44,249,869,7560,4120,800,1500,76,72,11.9,2,10922,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
772,Worcester State College,No,2197,1515,543,4,26,3089,2029,6797,3900,500,1200,60,60,21.0,14,4469,40
773,Xavier University,Yes,1959,1805,695,24,47,2849,1107,11520,4960,600,1250,73,75,13.3,31,9189,83
774,Xavier University of Louisiana,Yes,2097,1915,695,34,61,2793,166,6900,4200,617,781,67,75,14.4,20,8323,49
775,Yale University,Yes,10705,2453,1317,95,99,5217,83,19840,6510,630,2115,96,96,5.8,49,40386,99


**Transform categorical variable and create target/feature dataframes**

In [87]:
ohe = OneHotEncoder()
# fit and transform data
one_hot_encoded = ohe.fit_transform(df[['Private']])
# create a DataFrame from the transformed data
df_encoded = pd.DataFrame(one_hot_encoded.toarray(), columns=ohe.get_feature_names_out(['Private']))
# concatenate the original DataFrame with the one-hot encoded DataFrame
df_final = pd.concat([df, df_encoded], axis=1)


x = df_final.loc[:, df_final.columns.drop(['Apps', 'Unnamed: 0', 'Private',"Private_No"]) ]
y = df_final.loc[:, df_final.columns == 'Apps']



**Standardize target and feature variables** 

In [113]:
scale = pp.StandardScaler()

scale.fit(x)
xs = scale.transform(x)

scale.fit(y)
ys = scale.transform(y)

array([[-0.32120545, -0.0635089 , -0.2585828 , ..., -0.50191008,
        -0.31825194,  0.61255305],
       [-0.03870299, -0.28858421, -0.6556556 , ...,  0.16610985,
        -0.55126184,  0.61255305],
       [-0.37631793, -0.47812132, -0.31530749, ..., -0.17728996,
        -0.66776679,  0.61255305],
       ...,
       [-0.04237716, -0.0915087 ,  0.36538874, ..., -0.25624125,
        -0.95902917,  0.61255305],
       [ 0.17725626,  0.57833266,  3.82559456, ...,  5.88797079,
         1.9535946 ,  0.61255305],
       [-0.06687159, -0.09581636,  0.02504063, ..., -0.98711561,
         1.9535946 ,  0.61255305]])

**Split data into train and test groups**

In [70]:
xstrain,xstest, ystrain,ystest = model_selection.train_test_split(xs,ys, test_size=.30 )

**Linear Model**

In [86]:
linear = lm.LinearRegression().fit(xstrain, ystrain)
preds = linear.predict(xstest)
print("The R^2 value is: ", linear.score(xstrain,ystrain))

print("The Linear Regression model's MSE is: ", round(metrics.mean_squared_error(ystest,preds),4))


The R^2 value is:  0.9394677313094707
The Linear Regression model's MSE is:  0.079


**Ridge Regression**

In [109]:
ridge = lm.RidgeCV(cv=10, alphas=[.01,.1,1,10], scoring='neg_mean_squared_error').fit(xstrain, ystrain)
pred = ridge.predict(xstest)

print("The Ridge Model's R^2 value is ", ridge.score(xstrain,ystrain))



print("The Ridge Model's MSE is: ", round(metrics.mean_squared_error(ystest,pred),4))

print('The best lamda value is', ridge.alpha_)

The Ridge Model's R^2 value is  0.9394677266713616
The Ridge Model's MSE is:  0.0789
The best lamda value is 0.01


**Lasso Regression**

In [112]:
lasso = lm.LassoCV(cv=10, alphas=[.01,.1,1,10]).fit(xstrain, ystrain.ravel())
pred = lasso.predict(xstest)

lm.LassoCV()

print("The Lasso Models' R^2 value is: ", lasso.score(xstrain,ystrain))

print("The Lasso Model's MSE is: ", round(metrics.mean_squared_error(ystest,pred),4))


print('Array of all beta coefficients: ', lasso.coef_)

print('Array of non zero coefficients: ', lasso.coef_[lasso.coef_ != 0 ])

print('The best lamda value is', lasso.alpha_)



The Lasso Models' R^2 value is:  0.9322853896309471
The Lasso Model's MSE is:  0.0676
Array of all beta coefficients:  [ 0.90161149 -0.01167171  0.13531821 -0.         -0.         -0.
 -0.05512945  0.02260111 -0.          0.         -0.02394414 -0.01253449
  0.         -0.          0.10973844  0.0223559  -0.03555084]
Array of non zero coefficients:  [ 0.90161149 -0.01167171  0.13531821 -0.05512945  0.02260111 -0.02394414
 -0.01253449  0.10973844  0.0223559  -0.03555084]
The best lamda value is 0.01


# **Best Model: Lasso**#

- Lasso Model had the lowest MSE
- Suggests that there are a few variables that best explain college applications. 

# **Key Points:**#


- Tried 3 different regression models: One classic linear regression and two regularization regression techniques. 
- These methods were introduced to prevent overfitting and reduce magnitude of coefficients 



Ridge Regression: 


- Purpose: Create a model with a lower variance and lower complexity.
- Penalty term: square of the magnitude of the coefficients. (**L2 Norm**)
- shrink the coefficients towards zero, but not to set any of them to exactly zero


Lasso Regression: 
- Purpose: Create model with lower variance and higher interpretability
- Penalty term: absolute value of the magnitude of the coefficients (**L1 Norm**)
- Shrink some of the coefficients to zero, effectively performing feature selection and reducing the number of variables in the model. 


Hyperparameters:
- Lamda: Used to tune regularization in Ridge and Lasso. Adjusts the penalty given to beta coefficients
- Number of cross folds






